In [162]:
import sys
sys.path.append('..')
from tool import *
def fill_albedo(month,type,dir_name):
    max_distance=3
    distence_w_dim=[1,1,1,1]
    bands=['albedo_BSA_vis', 
        'albedo_WSA_vis', 
        'albedo_BSA_nir', 
        'albedo_WSA_nir', 
        'albedo_BSA_shortwave', 
        'albedo_WSA_shortwave']
    # band='albedo_BSA_vis'

    # m_index=[(month-3)%12+1,(month-2)%12+1,(month-1)%12+1,(month)%12+1,(month+1)%12+1]
    m_index=[(month-2)%12+1,(month-1)%12+1,(month)%12+1]
    offset=[1 if m_index[j]-m_index[0]>=0 else -1 for j in range(len(m_index))]
    if np.array(offset).sum()==len(offset):
        offset=[0]*len(m_index)
    # print(month,m_index,offset)

    vs=[]
    ms=[]
    land_proportions=[]
    for i in range(len(offset)):    
        NC=nc.Dataset(root_path+f'{dir_name}/month_{m_index[i]}.nc','r')
        for band in bands:
            vs.append(NC.variables[f'{band}-{type}'][:])
        if type!='ocean':
            ms.append(NC.variables[f'area-{type}'][:])
        # qa=NC.variables['qa'][:]
        area_grid=NC.variables['area-grid'][:]
        area_land=NC.variables['area-land'][:]
        land_proportions.append(area_land/area_grid)
        NC.close()
    offset_bands=[]
    for i in offset:
        offset_bands+=[i]*len(bands)
    if offset[0]!=0:
        nan_array=np.zeros((1,180,360))*np.nan
        vs=[np.concatenate((nan_array,vs[i]),axis=0) if offset_bands[i]==1 else np.concatenate((vs[i],nan_array),axis=0) for i in range(len(offset_bands))]
        land_proportions=[np.concatenate((nan_array,land_proportions[i]),axis=0) if offset[i]==1 else np.concatenate((land_proportions[i],nan_array),axis=0) for i in range(len(offset))]
        if type!='ocean':
            ms=[np.concatenate((nan_array,ms[i]),axis=0) if offset[i]==1 else np.concatenate((ms[i],nan_array),axis=0) for i in range(len(offset))]
        else:
            ms=land_proportions  
    else:
        if type=='ocean':
            ms=land_proportions  
    M,H,W=vs[0].shape
    v=np.stack(vs).reshape((len(offset),len(bands),M,H,W)).transpose((1,0,2,3,4))#(6, 3, 21, 180, 360)

    # vv=[v[:,i] for i in range(len(bands))]
    m=np.stack(ms)
    # plt.imshow(v[0,1,1])
    # plt.show()
    # land_proportion=np.stack(land_proportions)
    # print(month,m_index,v.shape)
    all_indices = np.indices(v[0].shape).reshape(4, -1).T
    if type!='ocean':
        interp_indices = all_indices[((v[0]==2000) & (m>0)).reshape(-1)]
    else:
        interp_indices = all_indices[((v[0]==2000) & (m<1) & (m>0)).reshape(-1)]
    nonan_indices=all_indices[((~np.isnan(v[0])).reshape(-1)) & ((v[0]!=2000).reshape(-1))]
    # print(len(nonan_indices))
    interp_indices=[j for j in interp_indices if j[0]==int(len(offset)/2)]   
    # interp_values=[]
    # search_n=1
    print(month,type,len(interp_indices),round(len(interp_indices)/(len(nonan_indices)+0.001)*100,2),'%')
    # for interp_indice in tqdm(interp_indices):
    #     k=TS_IDW_4d(interp_indice,max_distance=max_distance*search_n,distence_w_dim=distence_w_dim)
    #     while np.isnan(k):
    #         search_n*=2
    #         k=TS_IDW_4d(interp_indice,max_distance=max_distance*search_n,distence_w_dim=distence_w_dim)

    #     interp_values.append(k)
    kk=[search_nonan(v,interp_indice,max_distance=max_distance,distence_w_dim=distence_w_dim) for interp_indice in interp_indices]
    interp_values=np.array([a[0] for a in kk])
    dws=np.array([a[1] for a in kk])
    print('fill value '+str((1-np.isnan(interp_values).sum()/interp_values.shape[0])*100)+'%')
    dws_v=np.zeros(v.shape[1:])
    for z in range(len(interp_indices)):
        v[:,interp_indices[z][0],interp_indices[z][1],interp_indices[z][2],interp_indices[z][3]]=np.array(interp_values[z])
        dws_v[interp_indices[z][0],interp_indices[z][1],interp_indices[z][2],interp_indices[z][3]]=dws[z]
    # print(offset)
    # return v[:,int(len(offset)/2)],dws_v[int(len(offset)/2)]
    
    y_pd=[]
    m_pd=[]
    type_pd=[]
    lat_pd=[]
    lon_pd=[]
    d1=[]
    d2=[]
    d3=[]
    d4=[]
    sub_band1=[]
    sub_band2=[]
    sub_band3=[]
    sub_band4=[]
    sub_band5=[]
    sub_band6=[]
    for z in range(len(interp_indices)):
        a=kk[z]
        # for abc in a:
            
        #     print(abc)
        for j in range(len(a[2])):
            if offset[int(len(offset)/2)]>0:
                y_pd.append(interp_indices[z][1]+2000)
                if interp_indices[z][1]==0:
                    print(interp_indices[z])
                    print(v.shape)
                    raise('error')
            else:
                y_pd.append(interp_indices[z][1]+2001)
            m_pd.append(month)
            type_pd.append(type)
            lat_pd.append(90-interp_indices[z][2])
            lon_pd.append(interp_indices[z][3]-180)



            d1.append(a[2][j][0])
            d2.append(a[2][j][1])
            d3.append(a[2][j][2])
            d4.append(a[2][j][3])
            sub_band1.append(a[3][0][j])
            sub_band2.append(a[3][1][j])
            sub_band3.append(a[3][2][j])
            sub_band4.append(a[3][3][j])
            sub_band5.append(a[3][4][j])
            sub_band6.append(a[3][5][j])
            # print(a[3])
            # print(a[0])
    pd_dict={'y':y_pd,
            'm':m_pd,
            'type':type_pd,
            'lat':lat_pd,
            'lon':lon_pd,
            'd1':d1,
            'd2':d2,
            'd3':d3,
            'd4':d4,
            'sub_band1':sub_band1,
            'sub_band2':sub_band2,
            'sub_band3':sub_band3,
            'sub_band4':sub_band4,
            'sub_band5':sub_band5,
            'sub_band6':sub_band6}
    pd_all=pd.DataFrame.from_dict(pd_dict)
            
            
            
            
    
    if offset[int(len(offset)/2)]>=0:
        return v[:,int(len(offset)/2),offset[int(len(offset)/2)]:],dws_v[int(len(offset)/2),offset[int(len(offset)/2)]:],pd_all
    else:
        return v[:,int(len(offset)/2),:-1],dws_v[int(len(offset)/2),:-1],pd_all
def TS_IDW_4d(v,interp_indice,max_distance,distence_w_dim):
    
        value_bands=v[:,zero(interp_indice[0]-int(max_distance/distence_w_dim[0])):interp_indice[0]+int(max_distance/distence_w_dim[0])+1,
                zero(interp_indice[1]-int(max_distance/distence_w_dim[1])):interp_indice[1]+int(max_distance/distence_w_dim[1])+1,
                zero(interp_indice[2]-int(max_distance/distence_w_dim[2])):interp_indice[2]+int(max_distance/distence_w_dim[2])+1,
                zero(interp_indice[3]-int(max_distance/distence_w_dim[3])):interp_indice[3]+int(max_distance/distence_w_dim[3])+1]
        value_bands[value_bands==2000]=np.nan
        value=value_bands[0]
        non_nan_indices = np.where(~np.isnan(value))

        non_nan_points = np.array(list(zip(non_nan_indices[0], non_nan_indices[1], non_nan_indices[2],non_nan_indices[3])))
         
        if len(non_nan_points)==0:
                interpolated_values=[np.nan]
                non_nan_values_bands=[]
                non_nan_points2=[]
                dws=np.nan
        else:
                k=np.array([zero(-interp_indice[0]+int(max_distance/distence_w_dim[0]))-int(max_distance/distence_w_dim[0]),
                        zero(-interp_indice[1]+int(max_distance/distence_w_dim[1]))-int(max_distance/distence_w_dim[1]),
                        zero(-interp_indice[2]+int(max_distance/distence_w_dim[2]))-int(max_distance/distence_w_dim[2]),
                        zero(-interp_indice[3]+int(max_distance/distence_w_dim[3]))-int(max_distance/distence_w_dim[3])])
                # print(k)
                # print(non_nan_points)
                non_nan_points+=k
                distance_s = np.array([np.linalg.norm(np.array(non_nan_points[i])*np.array(distence_w_dim)) for i in range(len(non_nan_points))])
                if len(distance_s)<20:
                        interpolated_values=[np.nan]
                        non_nan_values_bands=[]
                        non_nan_points2=[]
                        dws=np.nan
                else:

                        
                        kth = np.partition(distance_s, 19)[19]
                        distance1=distance_s.copy()
                        distance1=distance1[distance1<=kth]
                        non_nan_points2=[non_nan_points[j] for j in range(len(distance_s)) if distance_s[j]<=kth]
                        weights = 1.0 / distance1**2
                        dws=np.sum(weights)
                        weights /= dws
                        interpolated_values=[]
                        non_nan_values_bands=[]
                        for i in range(value_bands.shape[0]):
                                distance=distance_s.copy()
                                value=value_bands[i]
                                non_nan_values = value[non_nan_indices]
                                non_nan_values2=np.array([non_nan_values[i] for i in range(len(distance)) if distance[i]<=kth])
                                non_nan_values_bands.append(non_nan_values2)
                                interpolated_value = np.sum(weights * non_nan_values2)
                                interpolated_values.append(interpolated_value)
        # print(non_nan_points2)
        # print(distance1)
        # raise()
        return interpolated_values,dws,non_nan_points2,non_nan_values_bands
    
def search_nonan(v,interp_indice,max_distance,distence_w_dim):
    search_n=1
    k,dws,non_nan_points2,non_nan_values_bands=TS_IDW_4d(v,interp_indice,max_distance=max_distance*search_n,distence_w_dim=distence_w_dim)
    while np.isnan(k[0]):
        search_n*=2
        k,dws,non_nan_points2,non_nan_values_bands=TS_IDW_4d(v,interp_indice,max_distance=max_distance*search_n,distence_w_dim=distence_w_dim)
    return k,dws,non_nan_points2,non_nan_values_bands
def zero(k):
    if k<0:
        return 0
    else:
        return k

In [129]:
#寻找验证点
from tool import *
aaa={'y':[],'m':[],'lon':[],'lat':[],'type':[]}
landtypes=['landtype'+str(i) for i in range(1,18)]
for y in range(2001,2021):
    print(y)
    for m in range(1,13):
        
        f=torch.load(root_path+f'information_temp/result3_{y}_{m}_all.pth')
        
        a=[]
        for k in f.keys():
            if f[k][0]==1 and f[k][1][-1]==f[k][1][-2]:
                a.append(k)
            
        random.shuffle(a)
        a=a[:400]
        aa=[]
        for i in a:
            tt=[landtypes[j] for j in range(17) if f[i][1][j]>2.5]
            for t in tt:
                aaa['y'].append(y)
                aaa['m'].append(m)
                aaa['lon'].append(int(i.split(',')[1]))
                aaa['lat'].append(int(i.split(',')[0]))
                aaa['type'].append(t)
landtypes=['snow']
for y in range(2001,2021):
    print(y)
    for m in range(1,13):
        
        f=torch.load(root_path+f'information2/result3_{y}_{m}_all.pth')
        
        a=[]
        for k in f.keys():
            if f[k][0]==1 and f[k][1][-4]>2.5:
                if not np.isnan(f[k][2][17,0]):
                    a.append(k)
            
        random.shuffle(a)
        a=a[:400]
        aa=[]
        for i in a:
            aaa['y'].append(y)
            aaa['m'].append(m)
            aaa['lon'].append(int(i.split(',')[1]))
            aaa['lat'].append(int(i.split(',')[0]))
            aaa['type'].append('snow')
aaa_pd=pd.DataFrame.from_dict(aaa)
aaa_pd.to_csv(root_path+'information_val/nan_coords2.csv')

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [157]:
#添加天顶角，删除天顶角大于85的验证点
sza_list=[]
sza_L_list=[]
for i in range(len(aaa_pd)):
    sza_L_list.append(sza_L[0,aaa_pd.iloc[i]['m']-1,90-aaa_pd.iloc[i]['lat'],0])
    sza_list.append(sza_ar[0,aaa_pd.iloc[i]['m']-1,90-aaa_pd.iloc[i]['lat'],0])
aaa_pd['sza']=sza_list
aaa_pd['sza_L']=sza_L_list
aaa_pd=aaa_pd[aaa_pd['sza_L']<2]
len(aaa_pd)


538390

In [158]:
#添加验证点的ground truth
aaa2=[]
# aaa_pd=pd.read_csv(root_path+'information_val/nan_coords.csv')
landtypes=['landtype'+str(i) for i in range(1,18)]+['snow']

bands=['albedo_BSA_vis', 
    'albedo_WSA_vis', 
    'albedo_BSA_nir', 
    'albedo_WSA_nir', 
    'albedo_BSA_shortwave', 
    'albedo_WSA_shortwave']

for m in range(1,13):
    print(m)
    NC=nc.Dataset(root_path+f'information_temp/month_{m}.nc','r')
    for landtype in landtypes:
        kk={}
        aa_type=aaa_pd[(aaa_pd.type==landtype) & (aaa_pd.m==m)]
        n=0
        for band in bands:
            img=NC.variables[f'{band}-{landtype}'][:]
            # aa_type=[[y-2001,90-i[0],i[1]+180] for i in aaa[y] if i[2]==landtype]
            kk[band]=[]
            for ind, row in aa_type.iterrows():
                kk[band].append(img[row['y']-2001,90-row['lat'],row['lon']+180])
            n+=1
            aa_type[f'main_band{n}']=kk[band]
        aaa2.append(aa_type)
    NC.close()
aaa_pd2=pd.concat(aaa2)
aaa_pd2.to_csv(root_path+'information_temp/nan_coords2.csv')

1
2
3
4
5
6
7
8
9
10
11
12


In [159]:
#数值设为2000
landtypes=['landtype'+str(i) for i in range(1,18)]+['snow']
bands=['albedo_BSA_vis', 
'albedo_WSA_vis', 
'albedo_BSA_nir', 
'albedo_WSA_nir', 
'albedo_BSA_shortwave',
'albedo_WSA_shortwave']
for m in range(1,13):
    print(m)
    NC=nc.Dataset(root_path+f'information_temp/month_{m}.nc','r+')
    for landtype in landtypes:
        for band in bands:
            img=NC.variables[f'{band}-{landtype}'][:]
            # aa_type=[[y-2001,90-i[0],i[1]+180] for i in aaa[y] if i[2]==landtype]
            aa_type=aaa_pd[(aaa_pd.type==landtype) & (aaa_pd.m==m)]
            for ind, row in aa_type.iterrows():
                img[row['y']-2001,90-row['lat'],row['lon']+180]=2000
            NC.variables[f'{band}-{landtype}'][:]=img
            NC.sync()
    NC.close()

1
2
3
4
5
6
7
8
9
10
11
12


In [163]:
#提取每个需要插值点周围距离最近的20个点的值，和4个方向的偏移量，各项异性暂时设置为（1，1，1，1）
all_pd_list=[]
for month in range(1,13):
    print(month)
    NC=nc.Dataset(root_path+f'information_temp/month_{month}.nc','r+')
    for type in landtypes:
        v_fill,dws,pd_all=fill_albedo(month,type,'information_temp')
        # NC.createVariable(f'DWS-{type}', 'f4', ("time", "latitude", "longitude"))
        # NC.variables[f'DWS-{type}'][:]=dws
        all_pd_list.append(pd_all)
    NC.close()

1
1 landtype1 457 0.16 %
fill value 100.0%
1 landtype2 1102 0.4 %
fill value 100.0%
1 landtype3 4 0.01 %
fill value 100.0%
1 landtype4 1129 0.33 %
fill value 100.0%
1 landtype5 882 0.25 %
fill value 100.0%
1 landtype6 697 0.47 %
fill value 100.0%
1 landtype7 1607 0.58 %
fill value 100.0%
1 landtype8 1448 0.29 %
fill value 100.0%
1 landtype9 1958 0.36 %
fill value 100.0%
1 landtype10 3135 0.46 %
fill value 100.0%
1 landtype11 1159 0.25 %
fill value 100.0%
1 landtype12 1802 0.46 %
fill value 100.0%
1 landtype13 1975 0.43 %
fill value 100.0%
1 landtype14 840 0.39 %
fill value 100.0%
1 landtype15 0 0.0 %
fill value nan%
1 landtype16 1930 0.43 %
fill value 100.0%
1 landtype17 1302 0.28 %
fill value 100.0%
1 snow 7859 1.0 %
fill value 100.0%
2
2 landtype1 513 0.16 %
fill value 100.0%
2 landtype2 1079 0.39 %
fill value 100.0%
2 landtype3 32 0.03 %
fill value 100.0%
2 landtype4 1191 0.33 %
fill value 100.0%
2 landtype5 934 0.25 %
fill value 100.0%
2 landtype6 608 0.41 %
fill value 100.0%
2 lan

In [164]:
#合并两个pd
all_pd_concat=pd.concat(all_pd_list)
all_pd_concat.to_csv(root_path+'information_temp/regression.csv')
aaa_pd2=pd.read_csv(root_path+'information_temp/nan_coords2.csv')
print(len(all_pd_concat),len(aaa_pd2))
aaaaa=pd.merge(aaa_pd2,all_pd_concat,how='outer')
print(len(aaaaa))
aaaaa.to_csv(root_path+'information_temp/regression.csv')

15307594 538390
15307594


In [170]:
from tool import *
data=pd.read_csv(root_path+'information_temp/regression.csv')
ws_all={}
landtypes=['snow']+['landtype'+str(i) for i in range(1,18)]
# landtypes=['snow']
for lt in landtypes:

    if lt=='landtype15':
        continue
    print(lt)
    data_lt=data[(data.type==lt)&(data.sza_L==0)]
    res=[]
    for band_id in range(1,7):
        res.append(np.array(data_lt[f'main_band{band_id}'])-np.array(data_lt[f'sub_band{band_id}']))
    d_offset=[]
    for d in range(1,5):
        d_offset.append(np.array(data_lt[f'd{d}']))
    d_offset=np.stack(d_offset)
    d_offset_sq=d_offset**2
    res=np.stack(res)
    res_sq=abs(res)#这里应该是**(1/q）,再**2
    ws_b=[]
    for band_i in range(6):
        x,y=d_offset_sq.transpose((1,0)).copy(), res_sq[band_i].copy()
        model = LinearRegression(fit_intercept=False).fit(x,y)#无截距且回归系数限制大于0
        ws=model.coef_
        ws2=[math.sqrt(ws[i]/ws[-1]) if ws[i]>=0 else -1 for i in range(4)]
        ws_b.append(ws2)
    ws_b=np.stack(ws_b)
        # if -1 in ws:
    ws_all[lt]=ws_b
    print(ws_b)
torch.save(ws_all,root_path+'information_temp/weights.pth')

snow
[[1.83375067 1.18553926 1.26304342 1.        ]
 [1.83272409 1.18545255 1.2633641  1.        ]
 [1.97179476 1.22108721 1.25441699 1.        ]
 [1.9024366  1.22124524 1.25242932 1.        ]
 [1.88207352 1.19532322 1.26208158 1.        ]
 [1.85598781 1.19499425 1.26139371 1.        ]]
landtype1
[[1.83678465 1.03761383 1.04681331 1.        ]
 [1.68022936 1.02260868 1.05219285 1.        ]
 [1.12999039 0.79824295 1.06662037 1.        ]
 [1.20029987 0.80874037 1.06631014 1.        ]
 [1.36347243 0.86718984 1.05858199 1.        ]
 [1.30247286 0.87209948 1.06113354 1.        ]]
landtype2
[[1.01214858 0.88493109 0.98704757 1.        ]
 [0.99072426 0.85282901 0.9906057  1.        ]
 [0.80768218 0.71972006 0.99447257 1.        ]
 [0.86553682 0.74221409 0.99471744 1.        ]
 [0.76644795 0.72084576 0.99462989 1.        ]
 [0.82800911 0.74570608 0.99481654 1.        ]]
landtype3
[[2.19911611 1.1125741  1.0335029  1.        ]
 [2.10439143 1.1011514  1.03733865 1.        ]
 [1.82326118 1.0514682